In [ ]:
import os
import shutil
import ffmpeg

# 1번과 2번 파일 경로
file1 = "/mnt/d/workspace/나인 솔즈 6화 (진 엔딩：후예사일(Shooting Star)).mkv"   # 첫 번째 파일 (MKV)
file2 = "/mnt/d/workspace/화면 녹화 중 2025-06-03 183906.mp4"   # 두 번째 파일 (MP4)

# file1을 자를 기준 타임스탬프 (HH:MM:SS)
split_ts = "02:51:26"

# 임시 폴더 생성
temp_dir = "temp_parts"
os.makedirs(temp_dir, exist_ok=True)

# 1) file1 앞구간: 0 ~ split_ts
part1 = os.path.join(temp_dir, "part1.mp4")
(
    ffmpeg
    .input(file1, ss=0, to=split_ts, fflags="+genpts", avoid_negative_ts="make_zero")
    .output(part1, c="copy", fflags="+genpts", avoid_negative_ts="make_zero")
    .run(overwrite_output=True)
)

# 2) file1 뒷구간: split_ts ~ 끝
part3 = os.path.join(temp_dir, "part3.mp4")
(
    ffmpeg
    .input(file1, ss=split_ts, fflags="+genpts", avoid_negative_ts="make_zero")
    .output(part3, c="copy", fflags="+genpts", avoid_negative_ts="make_zero")
    .run(overwrite_output=True)
)

# 3) file2를 그대로 part2로 복사 (이미 MP4이므로 컨테이너만 복사)
part2 = os.path.join(temp_dir, "part2.mp4")
shutil.copyfile(file2, part2)

# 4) concat_list.txt 생성 (절대 경로, single-quote로 감싸기)
concat_txt = os.path.join(temp_dir, "concat_list.txt")
with open(concat_txt, "w", encoding="utf-8") as f:
    for p in [part1, part2, part3]:
        f.write(f"file '{os.path.abspath(p)}'\n")

# 5) 세 파트를 A→B→C 순으로 이어붙이기 (concat 단계에도 genpts/avoid_negative_ts 지정)
merged = "merged.mp4"
(
    ffmpeg
    .input(concat_txt, format="concat", safe=0, fflags="+genpts", avoid_negative_ts="make_zero")
    .output(merged, c="copy", fflags="+genpts", avoid_negative_ts="make_zero")
    .run(overwrite_output=True)
)

print("✅ 편집 완료:", merged)
